In [20]:
#https://towardsdatascience.com/deep-transfer-learning-for-natural-language-processing-text-classification-with-universal-1a2c69e5baa9
import os, pandas as pd
import re
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [21]:
file_list=os.listdir('../../dataset/df_train.pkl.gz/')
df_train=pd.DataFrame()
for file in file_list:
    df_train=pd.concat([df_train, 
                        pd.read_pickle('../../dataset/df_train.pkl.gz/'+file, compression='gzip')])
df_train['mission_prgrm']=df_train['mission']+'; '+df_train['prgrm_dsc']

small_num=0
while small_num<100: # Make sure each category has at least 100 records.
    trainDF = df_train[df_train.mission.notna() & df_train.NTEE1.notna()].sample(40000)
    small_num=trainDF.groupby('NTEE1').count().sort_values('EIN').iloc[0]['EIN']
    
#### Sample ####
len(trainDF)

40000

In [22]:
#df_train['labels'] = df_train['NTEE1'].apply(lambda x:ord(x)-65)
#df_train.labels.head()

trainDF['labels'] = trainDF['NTEE1'].apply(lambda x:ord(x)-65)

In [23]:
texts = trainDF['mission_prgrm'].values
labels = trainDF['labels'].values

train_texts = texts[:30000]
train_labels = labels[:30000]

val_texts = texts[30000:35000]
val_labels = labels[30000:35000]

test_texts = texts[35000:]
test_labels = labels[35000:]

train_texts.shape, val_texts.shape, test_texts.shape

((30000,), (5000,), (5000,))

In [24]:
#text-preprocessing: Eliminating steps for contractions (We've --> We have)

import contractions
from bs4 import BeautifulSoup
import unicodedata
import re

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text):
    return contractions.fix(text)

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

def pre_process_document(document):
    # strip HTML
    #document = strip_html_tags(document)
    # lower case
    document = document.lower()
    # remove extra newlines (often might be present in really noisy text)
    #document = document.translate(document.maketrans("\n\t\r", "   "))
    # remove accented characters
    #document = remove_accented_chars(document)
    # expand contractions    
    #document = expand_contractions(document)  
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    #special_char_pattern = re.compile(r'([{.(-)!}])')
    #document = special_char_pattern.sub(" \\1 ", document)
    #document = remove_special_characters(document, remove_digits=True)  
    # remove extra whitespace
    #document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document


pre_process_corpus = np.vectorize(pre_process_document)

train_texts = pre_process_corpus(train_texts)
val_texts = pre_process_corpus(val_texts)
test_texts = pre_process_corpus(test_texts)

In [25]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': train_texts}, train_labels, 
    batch_size=256, num_epochs=None, shuffle=True)
    
# Prediction on the whole training set.
predict_train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': train_texts}, train_labels, shuffle=False)
    
# Prediction on the whole validation set.
predict_val_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': val_texts}, val_labels, shuffle=False)
    
# Prediction on the test set.
predict_test_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': test_texts}, test_labels, shuffle=False)  

In [26]:
#Encoders: https://tfhub.dev/s?module-type=text-embedding

embedding_feature = hub.text_embedding_column(
    key='sentence', 
    module_spec="https://tfhub.dev/google/universal-sentence-encoder/2",
    #module_spec="https://tfhub.dev/google/universal-sentence-encoder-large/3",
    trainable=False)

In [29]:
'''
dnn = tf.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=26,
          activation_fn=tf.nn.relu,
          dropout=0.1,
          optimizer=tf.train.AdagradOptimizer(learning_rate=0.005))

#73.6

dnn = tf.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=26,
          activation_fn=tf.nn.relu,
          dropout=0.3,
          optimizer=tf.train.AdagradOptimizer(learning_rate=0.01))
#75
'''
dnn = tf.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=26,
          activation_fn=tf.nn.relu,
          dropout=0.2,
          optimizer=tf.train.AdagradOptimizer(learning_rate=0.02))
#75.2

#TRY TANH Next

'''
dnn = tf.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=26,
          activation_fn=tf.nn.elu,
          dropout=0.3,
          optimizer=tf.train.AdagradOptimizer(learning_rate=0.02))
'''

'\ndnn = tf.estimator.DNNClassifier(\n          hidden_units=[512, 128],\n          feature_columns=[embedding_feature],\n          n_classes=26,\n          activation_fn=tf.nn.elu,\n          dropout=0.3,\n          optimizer=tf.train.AdagradOptimizer(learning_rate=0.02))\n'

In [30]:
tf.logging.set_verbosity(tf.logging.ERROR)
import time

TOTAL_STEPS = 1500   #1500
STEP_SIZE = 100   #100
for step in range(0, TOTAL_STEPS+1, STEP_SIZE):
    print()
    print('-'*100)
    print('Training for step =', step)
    start_time = time.time()
    dnn.train(input_fn=train_input_fn, steps=STEP_SIZE)
    elapsed_time = time.time() - start_time
    print('Train Time (s):', elapsed_time)
    print('Eval Metrics (Train):', dnn.evaluate(input_fn=predict_train_input_fn))
    print('Eval Metrics (Validation):', dnn.evaluate(input_fn=predict_val_input_fn))


----------------------------------------------------------------------------------------------------
Training for step = 0
Train Time (s): 103.63856220245361
Eval Metrics (Train): {'accuracy': 0.6708, 'average_loss': 1.1908067, 'loss': 152.01788, 'global_step': 100}
Eval Metrics (Validation): {'accuracy': 0.6722, 'average_loss': 1.1874663, 'loss': 148.43327, 'global_step': 100}

----------------------------------------------------------------------------------------------------
Training for step = 100
Train Time (s): 147.3950629234314
Eval Metrics (Train): {'accuracy': 0.69593334, 'average_loss': 1.0851234, 'loss': 138.5264, 'global_step': 200}
Eval Metrics (Validation): {'accuracy': 0.6952, 'average_loss': 1.0916027, 'loss': 136.45033, 'global_step': 200}

----------------------------------------------------------------------------------------------------
Training for step = 200
Train Time (s): 143.4216492176056
Eval Metrics (Train): {'accuracy': 0.7097, 'average_loss': 1.0391227, 'l

# 